In [56]:
import cv2 as cv
from scipy import ndimage
from skimage.util import random_noise
import numpy as np

class ObjectDetection:
    def __init__(self, img):
        #self.image_path = image_path
        self.img = img

    def shape(self):
        shape = self.img.shape
        if self.img is not None:
            print("Original Shape:", shape)
        else:
            print("Failed to read the image.")
            
    def resize(self, new_size):
        self.img = cv.resize(self.img, new_size)
        
        if isinstance(self.img, type(None)):
            print("Failed to resize the image. Image not resized.")
        else:
            
            print("Resized Shape:", self.img.shape)
    
    def rotate(self, angle): # improve later; include anti-clockwise option
        
        self.img = ndimage.rotate(self.img, angle)
        
        if isinstance(self.img, type(None)):
            print("Failed to rotate the image. Image not rotated.")
        else:
            
            print(f"Image rotated {angle} degrees") # this has to be changed later
    
    def noise_salt_pepper(self, amount):
        
        
        if isinstance(self.img, type(None)):
            print("Failed to add noise to the image.")
        else:
            noisy_img = random_noise(self.img, mode='s&p', amount=amount)
            noisy_img = np.array(255 * noisy_img, dtype='uint8')
            print(f"Salt & Pepper {amount} noise added")
    
    def noise_gaussian(self, var): # variance
        
        if isinstance(self.img, type(None)):
            print("Failed to add noise to the image.")
        else:
            gaussy_img = random_noise(self.img, mode='gaussian', var=var)
            gaussy_img = np.array(255 * gaussy_img, dtype='uint8')
            print(f"Gaussian noise with variance {var} added")
    
    def noise_speckle(self,var): # amount = 0.02, 0.05, 0.1, 0.2 etc
        
        if isinstance(self.img, type(None)):
            print("Failed to add noise to the image.")
        else:
            speckled_img = random_noise(self.img, mode='speckle', var=var)
            speckled_img = np.array(255 * speckled_img, dtype='uint8')
            print(f"Speckled noise with variance {var} added")
    
    def change_loop(self,new_size,new_angle,noise): # newsize is a tuple
        
        if isinstance(self.img, type(None)):
            print("Failed to modify the image.")
        else:
            original_img = self.img.copy() # for reseting
            self.img = cv.resize(original_img, new_size)
            self.img = ndimage.rotate(self.img, new_angle)
            
            if noise == 'Gaussian':               
                self.img =self.noise_gaussian(0.5)
            elif noise =='Salt & Pepper':
                self.img = self.noise_salt_pepper(0.5)
            elif noise == 'Speckle':
                self.img = self.noise_speckle(0.2)
            print("Image is rotated, resized and {noise} added.")
                
                
# Usage
image_path = "bermuda.jpg"
reader = ObjectDetection(cv.imread(img_path))
reader.shape()
reader.resize((416, 416))
reader.rotate(-90)
reader.noise_salt_pepper(0.5)
reader.noise_gaussian(0.5)
reader.noise_speckle(0.2)
reader.change_loop((220, 220), 180, 'Gaussian')

Original Shape: (2229, 3344, 3)
Resized Shape: (416, 416, 3)
Image rotated -90 degrees
Salt & Pepper 0.5 noise added
Gaussian noise with variance 0.5 added
Speckled noise with variance 0.2 added
Gaussian noise with variance 0.5 added
Image is rotated, resized and {noise} added.


shape (2229, 3344, 3)
